In [5]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import math

import collections
from collections import Counter

import nltk
from nltk import ConcordanceIndex

import spacy
nlp = spacy.load("en_core_web_md", disable = ['tagger', 'parser'])
vocab = nlp.vocab.strings


from string import punctuation
from re import sub
punctuation = punctuation +'”'+'“'+'’' + '—' + '’' + '‘' +'0123456789'

import progressbar

test_doc = open("./test_doc.txt", "r", encoding = "utf-8")
test_doc = test_doc.read()

In [43]:
n_vectors = 15000  # number of vectors to keep
removed_words = nlp.vocab.prune_vectors(n_vectors)

assert len(nlp.vocab.vectors) <= n_vectors  # unique vectors have been pruned
assert nlp.vocab.vectors.n_keys > n_vectors

In [44]:
vectors = [i for i in nlp.vocab.vectors]

In [46]:
len( nlp.vocab.vectors)

15000

In [2]:
lang = 'en'
pipeline = ['tagger', 'parser', 'ner']

In [48]:
bytes_data = nlp.to_bytes('reduced_model')
lang = nlp.meta["lang"]  # "en"
pipeline = nlp.meta["pipeline"]

In [4]:
nlp = spacy.blank(lang)
for pipe_name in pipeline:
    pipe = nlp.create_pipe(pipe_name)
    nlp.add_pipe(pipe)
nlp.from_bytes('reduced_model')

TypeError: a bytes-like object is required, not 'str'

In [4]:
stopwords = ['whom','hast','thou','therein', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']

In [5]:
def most_similar(word, topn=10):
    #https://stackoverflow.com/questions/57697374/list-most-similar-words-in-spacy-in-pretrained-model
    ms = nlp.vocab.vectors.most_similar(nlp(word).vector.reshape(1,nlp(word).vector.shape[0]), n=topn)
    words = [nlp.vocab.strings[w] for w in ms[0][0]]
    distances = ms[2]
    return words, distances

In [6]:

def latent_meaning_spacy(i, top_ = 10):
    '''
    INPUT: word tuple, topn
    
    OUTPUT: word tuple, the distance between the two original words, and the distance between the topn related words
    '''
    if(i[0] in vocab) & (i[1] in vocab):
        
        first_close, first_close_distances = most_similar(i[0], topn= top_)
        second_close, second_close_distances = most_similar(i[1], topn= top_)
        first_vec = nlp.vocab[i[0]].vector
        second_vec = nlp.vocab[i[1]].vector
        item_dis = np.dot(first_vec, second_vec)/(np.linalg.norm(first_vec)*np.linalg.norm(second_vec))
        
        for z in first_close:
            first_vec = first_vec + nlp.vocab[z].vector
        
        for z in second_close:
            second_vec = second_vec +  nlp.vocab[z].vector
        
        first_vec = first_vec - nlp.vocab[i[0]].vector
        second_vec = second_vec - nlp.vocab[i[1]].vector
        
        latent_dis = np.dot(first_vec, second_vec)/(np.linalg.norm(first_vec)*np.linalg.norm(second_vec))
        
        return([i, item_dis, latent_dis])
    else:
        return([None, None, None])


def latent_meaning(i, model3):
    
    if(i[0] in model3.wv.vocab) & (i[1] in model3.wv.vocab):
        first_close = list(model3.wv.most_similar(i[0], topn= 5))
        second_close = list(model3.wv.most_similar(i[1], topn= 5))
        
        first_vec = model3.wv.get_vector(i[0])
        second_vec = model3.wv.get_vector(i[1])
        
        item_dis = dot(first_vec, second_vec)/(norm(first_vec)*norm(second_vec))
        
        for z in first_close:
            first_vec = first_vec + model3.wv.get_vector(z[0])
        for n in second_close:
            second_vec = second_vec + model3.wv.get_vector(n[0])
        
        first_vec = first_vec - model3[i[0]]
        second_vec = second_vec - model3[i[1]]
        cos_sim = dot(first_vec, second_vec)/(norm(first_vec)*norm(second_vec))
        
        return([i, item_dis, cos_sim])
    

In [15]:
#PMI is defined as //pmi(r,c)=log(P(r,c)/(P(r)*P(c)))//, with P(r,c) being the
#probability of co-occurrence and P(r) and P(c) the probability of
#occurrence of two words (estimated via frequency)

#- I considered words as co-occurring if they occurred within a window
#of 5 words:
#no no yes yes yes yes target yes yes yes yes no no


def most_similar(word, topn=20):
    #https://stackoverflow.com/questions/57697374/list-most-similar-words-in-spacy-in-pretrained-model
    ms = nlp.vocab.vectors.most_similar(nlp(word).vector.reshape(1,nlp(word).vector.shape[0]), n=topn)
    ms = list(zip(ms[0][0], ms[1][0], ms[2][0]))
    print(ms)
    ms = [i for i in ms if str(nlp.vocab.strings[i[0]]).lower() != word]
    ms = [i for i in ms if str(nlp.vocab.strings[i[0]]).lower() + 's' != word]
    ms = [i for i in ms if str(nlp.vocab.strings[i[0]]).lower() + 'es' != word]
    words = [nlp.vocab.strings[w[0]] for w in ms]
    distances = [i[2] for i in ms]
    return words, distances


In [25]:
nlp2

{'Lock': ('lock', 1.0),
 'sourced': ('locally', 1.0),
 'swell': ('winds', 1.0),
 'Geez': ('Jeez', 1.0),
 'flops': ('flop', 1.0),
 'RE': ('Re', 1.0),
 'speeches': ('speech', 1.0),
 'likewise': ('Likewise', 1.0),
 'reservations': ('Hotel', 1.0),
 'Tigers': ('lions', 1.0),
 'Durant': ('Bryant', 1.0),
 'Possible': ('possible', 1.0),
 'terrific': ('Fantastic', 1.0),
 'spins': ('spin', 1.0),
 'simulate': ('mimic', 1.0),
 'sampling': ('samples', 1.0),
 'Tour': ('tour', 1.0),
 'Pikachu': ('Pokemon', 1.0),
 'midwest': ('Midwest', 1.0),
 'wisely': ('accordingly', 1.0),
 '8-10': ('6-8', 1.0),
 'watermelon': ('pineapple', 1.0),
 'lackluster': ('underwhelming', 1.0),
 'glimpse': ('peek', 1.0),
 'saddest': ('happiest', 1.0),
 'subscriber': ('subscribers', 1.0),
 'RIOT': ('riot', 1.0),
 'islam': ('Islam', 1.0),
 'PS1': ('PS2', 1.0),
 'survivability': ('dps', 1.0),
 'devastated': ('devastating', 1.0),
 'Trading': ('trading', 1.0),
 'operators': ('operator', 1.0),
 'Voldemort': ('Harry', 1.0),
 'stains

In [14]:
nlp.vocab

In [257]:
def make_ci(komyagin):
    txt = (nltk.Text(komyagin))
    return(ConcordanceIndex(txt))

#for some reason having difficulty subsetting by tokens instead of characters:: instead just use enough 
#characters to later be able to reliably get a 5 word radius

def get_context(ci, word, width=150, lines=100):
    
    half_width =  (width - len(word) - 2) // 2
    context = width // 4 # approx number of words of context
    num = 5
    results = []
    offsets = ci.offsets(word)
    
    if offsets:
        for i in offsets:
            query_word = ci._tokens[i]
  
            left_context = ci._tokens[max(0, i - context) : i]
            right_context = ci._tokens[i + 1 : i + context]
           
            left_print = " ".join(left_context)[-half_width:]
            right_print = " ".join(right_context)[:half_width]
                
            full_line_print = " ".join([left_print, query_word, right_print])
            
            results.append(full_line_print)
            
    return ([num, results])

def clean_text(txt_ls):
    
    translator = str.maketrans('','', sub('\#', '', punctuation))

    clean_txt_ls = []
    for i in txt_ls:
        n = i.split()
        str_ = ""
        for z in n:
            z = z.lower()
            s = z.translate(str.maketrans(translator))
            if s not in stopwords:
          #  print(s)
                str_ = str_ + " " + s
        clean_txt_ls.append(str_[1:])
        
    return(clean_txt_ls)

def clean_context(ci, target_word1, target_word2, window = 5):
    tox = []
    #gets context around a target word
    word_one_context = get_context(ci, target_word1)
   
     #takes this context and reshapes it into a -5 to + 5 window
    for i in word_one_context[1]:
        split_i = i.split()
        for z in split_i:
            tox.append(z)
            
    to_mend = list(enumerate(tox))
    
    mended_tox = []
    #modifications necessary to account for words at the start and end of corpus
    for z in to_mend:
        if z[1] == target_word1:
            if z[0] < (window):
                padded = tox[0:(z[0]+window)]
                mended_tox.append(padded)
            elif z[0] > (len(tox) - 1):
                padded = tox[(z[0] -window):(len(tox)-1)]
                mended_tox.append(padded)
            else: 
                padded = tox[(z[0] -window):(z[0] + window)]
                mended_tox.append(padded)
#reshaping list into format that can be more quickly processed    
    final_tox = []
    for i in mended_tox:
        for n in i:
            final_tox.append(n)
    return(final_tox)


#gathers all contexts of the word in results
def mutual_informativity(ci, target_word1, target_word2, total_count, window = 10):
   
    final_tox =  clean_context(ci, target_word1, target_word2, window = 10)
    
#this will return the count of target_word2 in the vicinity of target_word1
    prob_numx = Counter(final_tox)
    prob_num = prob_numx[target_word2]
    P_rc = prob_num/total_count
    P_r = (count(ci, target_word1))/total_count
    P_c = (count(ci, target_word2))/total_count
 #checking for indexing errors and overlap errors.
#fix to overlap is a bit of a hack fix, most notable problems are double counting 
#so if number of cooccurances is greater than number of times a word appears in 
#a document, then it replaces cooccurance with the total number of times
#except in the case that this is an odd number (since it's not being double-counted then,
#it's being double counted once and has another appearance)
#odd numbers, is P_c - 1/total. 
#this still may cause some errors
    if P_rc == 0:
     #   print(prob_numx)
        return ([target_word1, target_word2, "ERROR"])
    elif P_rc > P_c:
     #   print(target_word2)
        rounded = int(P_rc/P_c)
        nr = P_rc/P_c
        if (rounded > nr):
            P_n = P_c - 1/total_count
        else:
            P_n = P_c
        mutinf = math.log10(P_n/(P_r*P_c))
        #print([P_rc, P_c, P_r, P_n])
    elif P_rc > P_r:
        rounded = int(P_rc/P_r)
        nr = P_rc/P_r
        if (rounded > nr):
            P_n = P_c -1/total_count
        else:
            P_n = P_c
        mutinf = math.log10(P_n/(P_r*P_c))
        #print([P_rc, P_c, P_r, P_n])
    else:
        mutinf = math.log10(P_rc/(P_r*P_c))
    return ([target_word1, target_word2, mutinf])

def count(ci, word):
   
    offsets = ci.offsets(word)

    return len(offsets)

def pmi(text):
    '''
    iterates through and finds shit
    '''
    
    clean_doc = clean_text(text.split())
    total_count = len(clean_doc)
    
    test_ci = make_ci(clean_doc)
    
    pmi_list = []
    ordered_set_hold = []
    ordered_set = [i for i in clean_doc if i not in ordered_set_hold and len(i) > 0]

    
    index = 0
    #realistically only words that at some point occur in a 3 word window are really worth looking at esp with these short texts
    for i in enumerate(ordered_set):
        if i[0] < (len((ordered_set))-1):
            item = mutual_informativity(test_ci, i[1], ordered_set[i[0] + 1], total_count)
            if(item not in pmi_list) & (item[0] != item[1]):
                pmi_list.append(item)
        if i[0] < (len(ordered_set)-2):
            item = mutual_informativity(test_ci, i[1], ordered_set[i[0] + 2], total_count)
            if(item not in pmi_list) & (item[0] != item[1]):
                pmi_list.append(item)        
        if i[0] < (len(ordered_set)-3):
            item = mutual_informativity(test_ci, i[1], ordered_set[i[0] + 3], total_count)
            if(item not in pmi_list) & (item[0] != item[1]):
                pmi_list.append(item)
        else:
            return(pmi_list)
        
#just to allow for sorting by actual pmi index
def takeSecond(elem):
    #print(elem[2])
    return elem[2]

def pmi_high(pmi_output, n):
    cat = pmi_output
    cat.sort(key = takeSecond, reverse = True)
    return cat[:n]

def pmi_low(pmi_output, n):
    cat = pmi_output
    cat.sort(key = takeSecond, reverse = False)
    return cat[:n]

In [ ]:
def pair_words(text):
    '''
    iterates through and finds shit
    '''
    
    clean_doc = clean_text(text.split())
    
    words_pairs = []

    index = 0
    #realistically only words that at some point occur in a 3 word window are really worth looking at esp with these short texts
    for i in enumerate(clean_doc):
        if i[0] < (len((clean_doc))-1):
            item = (i[1], clean_doc[i[0] + 1])
            if(item not in words_pairs) & ((clean_doc[i[0] + 1], i[1]) not in word_pairs) & (item[0] != item[1]):
                words_pairs.append(item)
        if i[0] < (len(clean_doc)-2):
            item =  (i[1], clean_doc[i[0] + 2])
            if(item not in words_pairs) & ((clean_doc[i[0] + 2], i[1]) not in word_pairs) & (item[0] != item[1]):
                words_pairs.append(item)        
        if i[0] < (len(clean_doc)-3):
            item =  (i[1], ordered_set[i[0] + 3])
            if(item not in words_pairs) & ((clean_doc[i[0] + 3], i[1]) not in word_pairs) & (item[0] != item[1]):
                words_pairs.append(item)
        else:
            return(words_pairs)

In [270]:
pmi_test = pmi(test_doc)
bar = progressbar.ProgressBar(maxval=4890, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])


word_pairs = []
item_dis = []
latent_dis = []
bar.start()
x = 0
for i in pmi_high(pmi_test, 4890):
    if i != None:
        j, k , l = latent_meaning_spacy([i[0], i[1]])
        word_pairs.append(j)
        item_dis.append(k)
        latent_dis.append(l)
        bar.update(x + 1)
        x = x + 1
bar.finish()

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in float_scalars
  del sys.path[0]
[========================================================================] 100%


In [277]:
isinstance(test_doc, str)


True

In [271]:
latent_meanings = pd.DataFrame({
    
    'word_pairs': word_pairs,
    'item_dis': item_dis,
    'latent_dis': latent_dis
    
})

In [272]:
latent_meanings['difference']= latent_meanings['item_dis'] - latent_meanings['latent_dis']

In [275]:
latent_meanings[(latent_meanings['difference'] > .05) & (latent_meanings['item_dis'] > 0.05)].sort_values(by = 'difference', ascending = False)

,word_pairs,item_dis,latent_dis,difference
4752,"[whale, sperm]",0.406892,0.258778,0.148114
3619,"[sperm, whale]",0.406892,0.258778,0.148114
2112,"[new, york]",0.386062,0.240859,0.145203
292,"[pikes, wallers]",0.336901,0.213824,0.123077
1818,"[swiftness, letter]",0.173467,0.054868,0.118599
4423,"[whales, spouting]",0.271814,0.157196,0.114618
2532,"[prey, swallow]",0.436203,0.325043,0.111159
4887,"[sperm, whales]",0.357355,0.251056,0.106300
2210,"[time, ay]",0.087841,-0.016900,0.104742
1080,"[manner, mischievous]",0.391926,0.302750,0.089176


In [278]:

def generate_lexicals(input_value, n = 200):
    if isinstance(input_value, str):
        pmi_text = pmi(input_value)
    else:
        return('error')
    word_pairs = []
    item_dis = []
    latent_dis = []
    x = 0
    for i in pmi_high(pmi_text, n):
        if i != None:
            j, k , l = latent_meaning_spacy([i[0], i[1]])
            word_pairs.append(j)
            item_dis.append(k)
            latent_dis.append(l)
            #bar update
            
    latent_meanings = pd.DataFrame({
    
    'word_pairs': word_pairs,
    'item_dis': item_dis,
    'latent_dis': latent_dis
    
    })
    latent_meanings['difference']= latent_meanings['item_dis'] - latent_meanings['latent_dis']
    lexicals = latent_meanings[(latent_meanings['difference'] > .05) & (latent_meanings['item_dis'] > 0.05)].sort_values(by = 'difference', ascending = False)
    lexicals_temp = [str(i) for i in lexicals['word_pairs']]
    if len(lexicals_temp) > 0:
        return( '_'.join(lexicals_temp))
    else:
        return('no relevant pairs in this selection')

In [279]:
generate_lexicals(test_doc)

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in float_scalars
  del sys.path[0]


''